In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win24\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\win24\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
regex = re.compile('[^a-zA-Z0-9 ]')

In [4]:
def get_words(text):
    text_onlyletters = regex.sub("", text)
    words = text_onlyletters.lower().split()
    meaningful_words = [wordnet_lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(meaningful_words)

In [8]:
df = pd.read_json('Combined.json')
df['text'] = df["title"] + ' ' + df["description"] + ' ' + df["content"]
df['predicted_category'] = df['author']
df = df.drop(columns=['title','description','content','source','author','url','urlToImage','publishedAt'])
df.head()

,text,predicted_category
0,400 White Hackers Are Quietly Saving World Fro...,Sarthak Dogra
1,Good News! Bengaluru Doctor Claims To Be Close...,KC Archana
2,"Hrithik Roshan, Ajay Devgn Request Cured COVID...",Vatsala Devki Vats
3,"Large caps should be initial beneficiaries, sa...",ET CONTRIBUTORS
4,Hackers begin exploiting COVID-19 situation at...,"Bhragu Haritas, ET CIO"


In [9]:
for i, row in df.iterrows():
    ifor_val = get_words(str(row['text']))
    df.at[i,'text'] = ifor_val
df.head()

,text,predicted_category
0,400 white hacker quietly saving world covid19b...,Sarthak Dogra
1,good news bengaluru doctor claim close finding...,KC Archana
2,hrithik roshan ajay devgn request cured covid1...,Vatsala Devki Vats
3,large cap initial beneficiary say ravi gopalak...,ET CONTRIBUTORS
4,hacker begin exploiting covid19 situation ente...,"Bhragu Haritas, ET CIO"


In [10]:
df.to_json('forPredicting.json')

In [57]:
df2 = pd.DataFrame()
df = pd.read_csv('News_Category_Dataset_v2.csv',index_col=0)
df["text"] = df["headline"] + ' '+df["short_description"]
df = df.drop(columns=['authors','link','date','headline','short_description'])
df2 = df[df.category=='BUSINESS']
df2 = df2.append(df[df.category=='POLITICS'])
df2 = df2.append(df[df.category=='SCIENCE'])
df2 = df2.append(df[df.category=='TECH'])
df2 = df2.append(df[df.category=='ENVIRONMENT'])
df2 = df2.append(df[df.category=='WORLD NEWS'])
df2 = df2.append(df[df.category=='WORLDPOST'])
df2 = df2.append(df[df.category=='THE WORLDPOST'])
df = df2
df.head()
#df["predicted_category"] = df['Unnamed: 0']
#df = df.drop(columns=['Unnamed: 0','source','author','url','urlToImage','publishedAt','title','content','description'])
#df.head()
#df.to_csv("test.csv")

,category,text
87,BUSINESS,"U.S. Launches Auto Import Probe, China Vows To..."
337,BUSINESS,Starbucks Says Anyone Can Now Sit In Its Cafes...
603,BUSINESS,Seattle Passes Controversial New Tax On City's...
604,BUSINESS,Uber Ends Forced Arbitration In Individual Cas...
669,BUSINESS,"Chili's Hit By Data Breach, Credit And Debit C..."


In [58]:
df['category'].value_counts()

POLITICS         32739
BUSINESS          5937
THE WORLDPOST     3664
WORLDPOST         2579
SCIENCE           2178
WORLD NEWS        2177
TECH              2082
ENVIRONMENT       1323
Name: category, dtype: int64

In [59]:
for i, row in df.iterrows():
    if (row['category'] == 'WORLDPOST'):
        df.at[i,'category'] = 'GENERAL'
    elif (row['category'] == 'THE WORLDPOST'):
        df.at[i,'category'] = 'GENERAL'
    elif (row['category'] == 'WORLD NEWS'):
        df.at[i,'category'] = 'GENERAL'
    ifor_val = get_words(str(row['text']))
    df.at[i,'text'] = ifor_val

In [68]:
df['category'].value_counts()

POLITICS       32739
GENERAL         8420
BUSINESS        5937
SCIENCE         2178
TECH            2082
ENVIRONMENT     1323
Name: category, dtype: int64

In [72]:
df2 = pd.DataFrame()
df2 = df[df.category=='POLITICS'].sample(1000)
df2 = df2.append(df[df.category=='GENERAL'].sample(1000))
df2 = df2.append(df[df.category=='BUSINESS'].sample(1000))
df2 = df2.append(df[df.category=='SCIENCE'].sample(1000))
df2 = df2.append(df[df.category=='TECH'].sample(1000))
df2 = df2.append(df[df.category=='ENVIRONMENT'].sample(1000))
df = df2

,category,text
19228,POLITICS,cashless toll road proliferate rental car fee ...
20712,POLITICS,trump war truth trying destroy credibility eve...
51037,POLITICS,trump campaign float every defense book second...
109010,POLITICS,doug ducey v fred duval v barry hess nonpartis...
80436,POLITICS,seattle teacher halt protest honor 911 anniver...
...,...,...
149806,ENVIRONMENT,kemosabe chatty porcupine eats banana video ke...
155635,ENVIRONMENT,bear camera capture life eye alaskan animal vi...
178806,ENVIRONMENT,first day fall autumnal equinox arrives septem...
146519,ENVIRONMENT,climate change could turn greenland well green...


In [77]:
df['category'].value_counts()

TECH           1000
GENERAL        1000
SCIENCE        1000
BUSINESS       1000
ENVIRONMENT    1000
POLITICS       1000
Name: category, dtype: int64

In [ ]:
#df['topic_area'].value_counts()
#df = df[df.topic_area != 'tech']
df['topic_area'].value_counts()

In [78]:
df = df.reset_index(drop=True)
df.to_csv("forModelTraining.csv")

In [91]:
df = pd.read_csv('Evaluation Dataset.csv')
df['predicted_category'] = df['Unnamed: 0']
df['text'] = df["title"] + ' '+ df["description"] + ' ' + df["content"]
df = df.drop(columns=['Unnamed: 0'])
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category,predicted_category,text
0,The Hill,Rebecca Kheel,Navy hospital ship arrives in New York,A U.S. Navy hospital ship arrived in New York ...,https://thehill.com/policy/defense/490136-navy...,https://thehill.com/sites/default/files/usnsco...,2020-03-30T14:41:02Z,A U.S. Navy hospital ship arrived in New York ...,GENERAL,name,Navy hospital ship arrives in New York A U.S. ...
1,The Hill,J. Edward Moreno,WHO warns more young people dying from coronav...,Officials from the World Health Organization (...,https://thehill.com/policy/international/49104...,https://thehill.com/sites/default/files/corona...,2020-04-03T18:01:13Z,Officials from the World Health Organization (...,SCIENCE,name,WHO warns more young people dying from coronav...
2,The Hill,J. Edward Moreno,NJ governor orders all flags across the state ...,New Jersey Gov. Phil Murphy (D) signed an exec...,https://thehill.com/blogs/blog-briefing-room/4...,https://thehill.com/sites/default/files/murphy...,2020-04-03T20:59:33Z,New Jersey Gov. Phil Murphy (D) signed an exec...,POLITICS,name,NJ governor orders all flags across the state ...
3,The Hill,Marina Pitofsky,Florida reporter says she was blocked from Gov...,A Florida reporter in a bureau operated by two...,https://thehill.com/media/490026-florida-repor...,https://thehill.com/sites/default/files/desant...,2020-03-29T02:36:23Z,A Florida reporter in a bureau operated by two...,POLITICS,name,Florida reporter says she was blocked from Gov...
4,The Hill,Steve Clemons,The Hill's Coronavirus Report: NATO Deputy Sec...,Senate expected to pass relief bill today | Tr...,https://thehill.com/homenews/coronavirus-repor...,https://thehill.com/sites/default/files/corona...,2020-04-21T19:50:03Z,VIEW BREAKING NEWS ON CORONAVIRUS \r\nWHAT YOU...,POLITICS,name,The Hill's Coronavirus Report: NATO Deputy Sec...


In [87]:
df.to_csv("labeled.csv")

In [92]:
df = df.drop(columns=['title','description','content','source','author','url','urlToImage','publishedAt'])
df.head()

,category,predicted_category,text
0,GENERAL,name,Navy hospital ship arrives in New York A U.S. ...
1,SCIENCE,name,WHO warns more young people dying from coronav...
2,POLITICS,name,NJ governor orders all flags across the state ...
3,POLITICS,name,Florida reporter says she was blocked from Gov...
4,POLITICS,name,The Hill's Coronavirus Report: NATO Deputy Sec...


In [93]:
for i, row in df.iterrows():
    ifor_val = get_words(str(row['text']))
    df.at[i,'text'] = ifor_val

In [96]:
df.head()

,category,predicted_category,text
0,GENERAL,name,navy hospital ship arrives new york u navy hos...
1,SCIENCE,name,warns young people dying coronavirus official ...
2,POLITICS,name,nj governor order flag across state lowered ha...
3,POLITICS,name,florida reporter say blocked gov desantiss cor...
4,POLITICS,name,hill coronavirus report nato deputy secretary ...


In [98]:
df.to_csv("forEvaluating.csv")